In [34]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment

# Load full audio files
file_path1 = "../audio/test_clip_1.mp3"
file_path2 = "../audio/test_clip_2.mp3"
y1, sr1 = librosa.load(file_path1)
y2, sr2 = librosa.load(file_path2)

In [35]:
transition_duration = 5  # seconds
frames_A = int(transition_duration * sr1)
frames_B = int(transition_duration * sr2)
segment_A = y1[-frames_A:]
segment_B = y2[:frames_B]

In [36]:
def compute_complexity(y_segment, sr):
    onset_env = librosa.onset.onset_strength(y=y_segment, sr=sr)
    return np.std(onset_env)

complexity_A = compute_complexity(segment_A, sr1)
complexity_B = compute_complexity(segment_B, sr2)
print(f"Complexity A (end of song 1): {complexity_A:.4f}")
print(f"Complexity B (start of song 2): {complexity_B:.4f}")

Complexity A (end of song 1): 2.5075
Complexity B (start of song 2): 2.8112


In [37]:
hop_length = 512
if complexity_A < complexity_B:
    rate = complexity_B / complexity_A
    D = librosa.stft(segment_A)
    D_stretched = librosa.phase_vocoder(D, rate=rate, hop_length=hop_length)
    stretched_segment = librosa.istft(D_stretched, hop_length=hop_length)
    sf.write("transition_stretched.wav", stretched_segment, sr1)
    sf.write("other_segment.wav", segment_B, sr2)
    simpler_first = True
else:
    rate = complexity_A / complexity_B
    D = librosa.stft(segment_B)
    D_stretched = librosa.phase_vocoder(D, rate=rate, hop_length=hop_length)
    stretched_segment = librosa.istft(D_stretched, hop_length=hop_length)
    sf.write("transition_stretched.wav", stretched_segment, sr2)
    sf.write("other_segment.wav", segment_A, sr1)
    simpler_first = False

In [ ]:
stretched_audio = AudioSegment.from_wav("transition_stretched.wav")
other_audio = AudioSegment.from_wav("other_segment.wav")
crossfade = stretched_audio.append(other_audio, crossfade=3000)
crossfade.export("../audio/transition_output.mp3", format="mp3")

<_io.BufferedRandom name='../audio/transition_output.mp3'>

In [ ]:
import librosa
import numpy as np
import soundfile as sf
from pydub import AudioSegment

# Load full First Song
full_song1 = AudioSegment.from_file("../audio/song1.wav")

if simpler_first:
    # === Smooth tempo recovery after transition ===
    y2, sr2 = librosa.load("../audio/song2_stretched.wav")
    y2_original, _ = librosa.load("../audio/test_clip_2.mp3")

    # First 5s of stretched segment already used in crossfade
    stretched_intro = y2[:int(sr2 * 5)]

    # Tempo ramp from `rate` → 1.0 over next 5 seconds
    recovery_segments = []
    recovery_duration = 5  # seconds
    recovery_ratios = np.linspace(rate, 1.0, recovery_duration)

    for i, r in enumerate(recovery_ratios):
        start = int(sr2 * (5 + i))
        end = int(sr2 * (6 + i))
        segment = y2[start:end]
        if len(segment) == 0:
            break
        recovered = librosa.effects.time_stretch(segment, rate=r)
        recovery_segments.append(recovered)

    y_recovery = np.concatenate(recovery_segments)

    # Remaining unaltered part of song2 (starting at 10s)
    y_rest = y2_original[int(sr2 * 10):]

    # Final merged signal
    y2_final = np.concatenate([stretched_intro, y_recovery, y_rest])
    sf.write("../audio/song2_tempo_restored.wav", y2_final, sr2)

    # Load and trim first 5s already used in crossfade
    full_song2 = AudioSegment.from_file("../audio/song2_tempo_restored.wav")
    song2_trimmed = full_song2[5000:]  # remove first 5 seconds
    final_mix = full_song1 + crossfade + song2_trimmed
else:
    # If Song 1 was stretched and part used in crossfade, trim last 5s
    full_song2 = AudioSegment.from_file("../audio/song2_stretched.wav")
    song1_cut = full_song1[:-5000]
    final_mix = song1_cut + crossfade + full_song2

# Export final mix
final_mix.export("../audio/final_complexity_mix.mp3", format="mp3")
print("✅ Final mix saved to audio/final_complexity_mix.mp3")

✅ Final mix saved to audio/final_complexity_mix.mp3
